In [67]:
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
import pandas as pd
import numpy as np
import sklearn
import pandas as pd
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [68]:
spark = SparkSession.builder.master('local[2]').config("spark.executor.memory","1g").config("spark.drive.memory","1g").appName('spark_sh_data').getOrCreate()

In [69]:
df = spark.read.options(header=None, nullValue='NULL', inferSchema=True).option('nullValue','null').csv('T4smallsubset.csv')

In [70]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
newDf = df.withColumn('_c4', translate('_c4', 'NULL', '')).withColumn('_c5', translate('_c5', 'NULL', '')).withColumn('_c6', translate('_c6', 'NULL', '')).withColumn('_c7', translate('_c7', 'NULL', ''))\
    .withColumn('_c8', translate('_c8', 'NULL', ''))
newDf1 = newDf.withColumn('_c4', newDf["_c4"].cast(IntegerType())).withColumn('_c5', newDf["_c5"].cast(IntegerType())).withColumn('_c6', newDf["_c6"].cast(IntegerType())).withColumn('_c7', newDf["_c7"].cast(IntegerType()))\
    .withColumn('_c8', newDf["_c8"].cast(IntegerType()))
newDf1.show()
newDf1.printSchema()

+----------+-------------+---+-----+----+----+----+----+----+--------+---------+--------+--------+----+----+----+----+----+----+----+----+----+----+----+----+--------------------+
|       _c0|          _c1|_c2|  _c3| _c4| _c5| _c6| _c7| _c8|     _c9|     _c10|    _c11|    _c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|                _c25|
+----------+-------------+---+-----+----+----+----+----+----+--------+---------+--------+--------+----+----+----+----+----+----+----+----+----+----+----+----+--------------------+
|0a50e09262|1451606401243|2.0|  2.4|null|null|null|null|null|    null|     null|    null|    null|null|null|null|null|null|null|null|null|null|null|null|null|                null|
|0a50e09262|1451606691043|2.0|  2.4|   0|   0|   0|   0|   0| 8289872|134217728|29090442|20800570|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|2016-01-01 02:04:...|
|0a50e09262|1451607965843|2.0|  2.4|  17|   0|   0|   0|   4| 3736234|134217728|34619480|30883246|  

In [71]:
from pyspark.sql.functions import rand,when

data= newDf1.withColumn('rand', when(rand() > 0.5, 'yes').otherwise('no'))

In [72]:
stringIndexer = StringIndexer().setInputCol("_c25").setOutputCol("_c25_index").setHandleInvalid("skip")
_c25_index_model=stringIndexer.fit(data)
_c25_index_df=_c25_index_model.transform(data)
_c25_index_df.toPandas().head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c18,_c19,_c20,_c21,_c22,_c23,_c24,_c25,rand,_c25_index
0,0a50e09262,1451606691043,2.0,2.4,0.0,0.0,0.0,0.0,0.0,8289872,...,0,0,0,0,0,0,0,2016-01-01 02:04:50.877,no,57.0
1,0a50e09262,1451607965843,2.0,2.4,17.0,0.0,0.0,0.0,4.0,3736234,...,0,0,0,0,0,0,0,2016-01-01 02:26:05.731,no,312.0
2,0a50e09262,1451611415651,2.0,2.4,NaN,NaN,NaN,NaN,NaN,3460158,...,0,0,0,0,0,0,0,2016-01-01 03:23:35.539,yes,1001.0
3,0a50e09262,1451608415819,2.0,2.4,NaN,NaN,NaN,NaN,NaN,1558612,...,1,129,2,78,1,129,2,2016-01-01 02:33:35.679,yes,402.0
4,0a50e09262,1451608030871,2.0,2.4,NaN,NaN,NaN,NaN,NaN,10359070,...,0,0,0,0,0,0,0,2016-01-01 02:27:10.762,no,325.0


In [73]:
encoder = OneHotEncoder().setInputCols(["_c25_index"]).setOutputCols(["_c25_encoded"])

encoder_model=encoder.fit(_c25_index_df)
encoder_df=encoder_model.transform(_c25_index_df)

encoder_df.toPandas().head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c19,_c20,_c21,_c22,_c23,_c24,_c25,rand,_c25_index,_c25_encoded
0,0a50e09262,1451606691043,2.0,2.4,0.0,0.0,0.0,0.0,0.0,8289872,...,0,0,0,0,0,0,2016-01-01 02:04:50.877,no,57.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0a50e09262,1451607965843,2.0,2.4,17.0,0.0,0.0,0.0,4.0,3736234,...,0,0,0,0,0,0,2016-01-01 02:26:05.731,no,312.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0a50e09262,1451611415651,2.0,2.4,NaN,NaN,NaN,NaN,NaN,3460158,...,0,0,0,0,0,0,2016-01-01 03:23:35.539,yes,1001.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0a50e09262,1451608415819,2.0,2.4,NaN,NaN,NaN,NaN,NaN,1558612,...,129,2,78,1,129,2,2016-01-01 02:33:35.679,yes,402.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0a50e09262,1451608030871,2.0,2.4,NaN,NaN,NaN,NaN,NaN,10359070,...,0,0,0,0,0,0,2016-01-01 02:27:10.762,no,325.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [74]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler().setInputCols(['_c4','_c5','_c6','_c7','_c8','_c9','_c10','_c11','_c12','_c13','_c14','_c15','_c16','_c17','_c18','_c19','_c20','_c21','_c22','_c23','_c24'])\
                    .setOutputCol("vectorized_features")\
                        .setHandleInvalid("skip")
                                  
assembler_df = assembler.transform(encoder_df)
assembler_df.toPandas().head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c20,_c21,_c22,_c23,_c24,_c25,rand,_c25_index,_c25_encoded,vectorized_features
0,0a50e09262,1451606691043,2.0,2.4,0,0,0,0,0,8289872,...,0,0,0,0,0,2016-01-01 02:04:50.877,no,57.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 8289872.0, 134217728..."
1,0a50e09262,1451607965843,2.0,2.4,17,0,0,0,4,3736234,...,0,0,0,0,0,2016-01-01 02:26:05.731,no,312.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(17.0, 0.0, 0.0, 0.0, 4.0, 3736234.0, 13421772..."
2,0a50e09262,1451609770986,2.0,1.4,35,0,0,0,8,32466,...,0,0,0,0,0,2016-01-01 02:56:10.88,no,672.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(35.0, 0.0, 0.0, 0.0, 8.0, 32466.0, 134217728...."
3,0a50e09262,1451608601266,2.0,0.3,13,0,0,0,3,9853730,...,0,0,0,0,0,2016-01-01 02:36:41.155,yes,439.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(13.0, 0.0, 0.0, 0.0, 3.0, 9853730.0, 13421772..."
4,0a50e09262,1451606446297,2.0,0.3,11,0,0,0,2,10645694,...,0,0,0,0,0,2016-01-01 02:00:46.161,yes,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(11.0, 0.0, 0.0, 0.0, 2.0, 10645694.0, 1342177..."


In [75]:
label_indexer = StringIndexer().setInputCol("rand").setOutputCol("label")

label_indexer_model=label_indexer.fit(assembler_df)
label_indexer_df = label_indexer_model.transform(assembler_df)

label_indexer_df.select("rand","label").toPandas().head()

,rand,label
0,no,0.0
1,no,0.0
2,yes,1.0
3,yes,1.0
4,no,0.0


In [76]:
scaler = StandardScaler().setInputCol("vectorized_features").setOutputCol("features")
scaler_model=scaler.fit(label_indexer_df)
scaler_df = scaler_model.transform(label_indexer_df)
pd.set_option('display.max_colwidth', 100)
scaler_df.select("vectorized_features","features").toPandas().head()

,vectorized_features,features
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 8289872.0, 134217728.0, 29090442.0, 20800570.0, 0.0, 0.0, 0.0, 0.0, 0....","(0.0, 0.0, 0.0, 0.0, 0.0, 2.368964603686052, 0.0, 1.3712824528073548, 0.9540789824449748, 0.0, 0..."
1,"(17.0, 0.0, 0.0, 0.0, 4.0, 3736234.0, 134217728.0, 34619480.0, 30883246.0, 0.0, 0.0, 0.0, 0.0, 0...","(0.9473679756432686, 0.0, 0.0, 0.0, 0.3336051891155314, 1.067689114752116, 0.0, 1.63191351473157..."
2,"(35.0, 0.0, 0.0, 0.0, 8.0, 32466.0, 134217728.0, 31711276.0, 31678810.0, 0.0, 0.0, 0.0, 0.0, 0.0...","(1.9504634792655529, 0.0, 0.0, 0.0, 0.6672103782310628, 0.0092776830357901, 0.0, 1.4948248752951..."
3,"(13.0, 0.0, 0.0, 0.0, 3.0, 9853730.0, 134217728.0, 34230824.0, 24377094.0, 0.0, 0.0, 0.0, 0.0, 0...","(0.7244578637272053, 0.0, 0.0, 0.0, 0.25020389183664854, 2.815862245433869, 0.0, 1.6135928184362..."
4,"(11.0, 0.0, 0.0, 0.0, 2.0, 10645694.0, 134217728.0, 31261076.0, 20615382.0, 0.0, 0.0, 0.0, 0.0, ...","(0.6130028077691738, 0.0, 0.0, 0.0, 0.1668025945577657, 3.0421787293788105, 0.0, 1.4736030815440..."


In [78]:
pipeline_stages=Pipeline().setStages([stringIndexer,encoder,assembler,label_indexer,scaler])

pipeline_model=pipeline_stages.fit(data)
pipeline_df=pipeline_model.transform(data)

pipeline_df.toPandas().head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c22,_c23,_c24,_c25,rand,_c25_index,_c25_encoded,vectorized_features,label,features
0,0a50e09262,1451606691043,2.0,2.4,0,0,0,0,0,8289872,...,0,0,0,2016-01-01 02:04:50.877,no,57.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 8289872.0, 134217728.0, 29090442.0, 20800570.0, 0.0, 0.0, 0.0, 0.0, 0....",0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 2.368964603686052, 0.0, 1.3712824528073548, 0.9540789824449748, 0.0, 0..."
1,0a50e09262,1451607965843,2.0,2.4,17,0,0,0,4,3736234,...,0,0,0,2016-01-01 02:26:05.731,no,312.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(17.0, 0.0, 0.0, 0.0, 4.0, 3736234.0, 134217728.0, 34619480.0, 30883246.0, 0.0, 0.0, 0.0, 0.0, 0...",0.0,"(0.9473679756432686, 0.0, 0.0, 0.0, 0.3336051891155314, 1.067689114752116, 0.0, 1.63191351473157..."
2,0a50e09262,1451609770986,2.0,1.4,35,0,0,0,8,32466,...,0,0,0,2016-01-01 02:56:10.88,no,672.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(35.0, 0.0, 0.0, 0.0, 8.0, 32466.0, 134217728.0, 31711276.0, 31678810.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0.0,"(1.9504634792655529, 0.0, 0.0, 0.0, 0.6672103782310628, 0.0092776830357901, 0.0, 1.4948248752951..."
3,0a50e09262,1451608601266,2.0,0.3,13,0,0,0,3,9853730,...,0,0,0,2016-01-01 02:36:41.155,yes,439.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(13.0, 0.0, 0.0, 0.0, 3.0, 9853730.0, 134217728.0, 34230824.0, 24377094.0, 0.0, 0.0, 0.0, 0.0, 0...",1.0,"(0.7244578637272053, 0.0, 0.0, 0.0, 0.25020389183664854, 2.815862245433869, 0.0, 1.6135928184362..."
4,0a50e09262,1451606446297,2.0,0.3,11,0,0,0,2,10645694,...,0,0,0,2016-01-01 02:00:46.161,yes,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(11.0, 0.0, 0.0, 0.0, 2.0, 10645694.0, 134217728.0, 31261076.0, 20615382.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,"(0.6130028077691738, 0.0, 0.0, 0.0, 0.1668025945577657, 3.0421787293788105, 0.0, 1.4736030815440..."


In [79]:
train, test = pipeline_df.randomSplit([0.8,0.2], seed=56)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 7520
Test Dataset Count: 1787


In [80]:
#Logistic Regression
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=5)
lrModel = lr.fit(train)
predictions = lrModel.transform(test)
#predictions_train = lrModel.transform(train)
predictions.select('label', 'features', 'rawPrediction', 'prediction', 'probability').toPandas().head()

,label,features,rawPrediction,prediction,probability
0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.8894453515307853, 0.0, 1.4704931601292144, 1.2880909544806276, 0.0, ...","[-0.04922577688392289, 0.04922577688392289]",1.0,"[0.48769604023482177, 0.5123039597651782]"
1,1.0,"(0.6130028077691738, 0.0, 0.0, 0.0, 0.1668025945577657, 3.0421787293788105, 0.0, 1.4736030815440...","[0.0069208253840407526, -0.0069208253840407526]",0.0,"[0.50173019943995, 0.49826980056005]"
2,0.0,"(0.33436516787409476, 0.0, 0.0, 1.0496025671002176, 0.5838090809521799, 0.4233002181550309, 0.0,...","[-0.049377372204952544, 0.049377372204952544]",1.0,"[0.48765816442490906, 0.5123418355750909]"
3,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 3.55954686924749, 0.0, 1.4770399562272487, 0.8658869802763666, 0.0, 0....","[0.009135115517774638, -0.009135115517774638]",0.0,"[0.5022837629977334, 0.49771623700226664]"
4,0.0,"(0.33436516787409476, 0.0, 0.0, 0.0, 0.08340129727888285, 1.8311298938973548, 0.0, 1.36423693041...","[-0.020500658088836114, 0.020500658088836114]",1.0,"[0.49487501496930386, 0.5051249850306961]"


In [81]:
predictions.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|  427|
|       1.0| 1360|
+----------+-----+



In [83]:
train.groupby("rand").count().show()

+----+-----+
|rand|count|
+----+-----+
|  no| 3720|
| yes| 3800|
+----+-----+



In [86]:
import pyspark.ml.evaluation as ev

evaluator = ev.BinaryClassificationEvaluator(rawPredictionCol='probability', labelCol='label')

print(evaluator.evaluate(predictions, {evaluator.metricName: 'areaUnderROC'}))
print(evaluator.evaluate(predictions, {evaluator.metricName: 'areaUnderPR'}))

0.5045719400470359
0.4918448255770519


In [88]:
#DECISION TREE
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtmodel = dt.fit(train)
dt_pred = dtmodel.transform(test)
dt_pred.select('label', 'features', 'rawPrediction', 'prediction', 'probability').toPandas().head()

,label,features,rawPrediction,prediction,probability
0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.8894453515307853, 0.0, 1.4704931601292144, 1.2880909544806276, 0.0, ...","[1098.0, 1268.0]",1.0,"[0.4640743871513102, 0.5359256128486898]"
1,1.0,"(0.6130028077691738, 0.0, 0.0, 0.0, 0.1668025945577657, 3.0421787293788105, 0.0, 1.4736030815440...","[1364.0, 1441.0]",1.0,"[0.48627450980392156, 0.5137254901960784]"
2,0.0,"(0.33436516787409476, 0.0, 0.0, 1.0496025671002176, 0.5838090809521799, 0.4233002181550309, 0.0,...","[1098.0, 1268.0]",1.0,"[0.4640743871513102, 0.5359256128486898]"
3,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 3.55954686924749, 0.0, 1.4770399562272487, 0.8658869802763666, 0.0, 0....","[1364.0, 1441.0]",1.0,"[0.48627450980392156, 0.5137254901960784]"
4,0.0,"(0.33436516787409476, 0.0, 0.0, 0.0, 0.08340129727888285, 1.8311298938973548, 0.0, 1.36423693041...","[1364.0, 1441.0]",1.0,"[0.48627450980392156, 0.5137254901960784]"


In [89]:
dt_pred.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|  573|
|       1.0| 1214|
+----------+-----+



In [90]:
import pyspark.ml.evaluation as ev

evaluator = ev.BinaryClassificationEvaluator(rawPredictionCol='probability', labelCol='label')

print(evaluator.evaluate(dt_pred, {evaluator.metricName: 'areaUnderROC'}))
print(evaluator.evaluate(dt_pred, {evaluator.metricName: 'areaUnderPR'}))

0.48341151722720044
0.4606040206848951


In [91]:
# Random Forest Classifier
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rfModel = rf.fit(train)
rf_pred = rfModel.transform(test)
rf_pred.select('label', 'features', 'rawPrediction', 'prediction', 'probability').toPandas().head()

,label,features,rawPrediction,prediction,probability
0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.8894453515307853, 0.0, 1.4704931601292144, 1.2880909544806276, 0.0, ...","[9.365118007241083, 10.634881992758917]",1.0,"[0.46825590036205417, 0.5317440996379459]"
1,1.0,"(0.6130028077691738, 0.0, 0.0, 0.0, 0.1668025945577657, 3.0421787293788105, 0.0, 1.4736030815440...","[9.899197335189664, 10.100802664810336]",1.0,"[0.49495986675948317, 0.5050401332405168]"
2,0.0,"(0.33436516787409476, 0.0, 0.0, 1.0496025671002176, 0.5838090809521799, 0.4233002181550309, 0.0,...","[9.767501592925596, 10.232498407074404]",1.0,"[0.4883750796462798, 0.5116249203537202]"
3,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 3.55954686924749, 0.0, 1.4770399562272487, 0.8658869802763666, 0.0, 0....","[10.338223022877921, 9.661776977122077]",0.0,"[0.5169111511438961, 0.48308884885610387]"
4,0.0,"(0.33436516787409476, 0.0, 0.0, 0.0, 0.08340129727888285, 1.8311298938973548, 0.0, 1.36423693041...","[9.961008416905228, 10.038991583094772]",1.0,"[0.4980504208452614, 0.5019495791547386]"


In [ ]:
rf_pred.groupBy("prediction").count().show()

In [92]:
import pyspark.ml.evaluation as ev

evaluator = ev.BinaryClassificationEvaluator(rawPredictionCol='probability', labelCol='label')

print(evaluator.evaluate(rf_pred, {evaluator.metricName: 'areaUnderROC'}))
print(evaluator.evaluate(rf_pred, {evaluator.metricName: 'areaUnderPR'}))

0.4954023608109398
0.48216467371233146


In [93]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(maxIter=10)
gbtModel = gbt.fit(train)
gbt_pred = gbtModel.transform(test)
gbt_pred.select('label', 'features', 'rawPrediction', 'prediction', 'probability').toPandas().head()

,label,features,rawPrediction,prediction,probability
0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.8894453515307853, 0.0, 1.4704931601292144, 1.2880909544806276, 0.0, ...","[-0.09607902225483159, 0.09607902225483159]",1.0,"[0.4521077655160538, 0.5478922344839462]"
1,1.0,"(0.6130028077691738, 0.0, 0.0, 0.0, 0.1668025945577657, 3.0421787293788105, 0.0, 1.4736030815440...","[-0.05712408441094266, 0.05712408441094266]",1.0,"[0.47146898481095445, 0.5285310151890455]"
2,0.0,"(0.33436516787409476, 0.0, 0.0, 1.0496025671002176, 0.5838090809521799, 0.4233002181550309, 0.0,...","[-0.007888307734246995, 0.007888307734246995]",1.0,"[0.49605592793968956, 0.5039440720603104]"
3,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 3.55954686924749, 0.0, 1.4770399562272487, 0.8658869802763666, 0.0, 0....","[0.14049684104423177, -0.14049684104423177]",0.0,"[0.5697898214874796, 0.4302101785125204]"
4,0.0,"(0.33436516787409476, 0.0, 0.0, 0.0, 0.08340129727888285, 1.8311298938973548, 0.0, 1.36423693041...","[-0.038563351822747595, 0.038563351822747595]",1.0,"[0.4807278765394516, 0.5192721234605484]"


In [95]:
gbt_pred.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|  611|
|       1.0| 1176|
+----------+-----+



In [96]:
import pyspark.ml.evaluation as ev

evaluator = ev.BinaryClassificationEvaluator(rawPredictionCol='probability', labelCol='label')

print(evaluator.evaluate(gbt_pred, {evaluator.metricName: 'areaUnderROC'}))
print(evaluator.evaluate(gbt_pred, {evaluator.metricName: 'areaUnderPR'}))

0.4942929100454814
0.47239426217462177
